# A common interface for handling tabular data

As we've seen in the FITS tutorial, the [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) sub-package can be used to access FITS tables. In addition, as we will see in the next tutorial, there is functionality in [astropy.io.votable](http://docs.astropy.org/en/stable/io/votable/index.html) and [astropy.io.ascii](http://docs.astropy.org/en/stable/io/ascii/index.html) to read in VO and ASCII tables. However, while these sub-pacakges have user interfaces that are specific to each kind of file, it can be difficult to remember all of them. Therefore, astropy includes a higher level interface in [astropy.table](http://docs.astropy.org/en/stable/table/index.html) which can be used to access tables in many different formats in a similar way.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create tables</li>
<li>Access data in tables</li>
<li>Combining tables</li>
<li>Using high-level objects as columns</li>
<li>Aggregation</li>
<li>Masking</li>
<li>Reading/writing</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.table. For more information about the features presented below as well as other available features, you can read the
[astropy.table documentation](https://docs.astropy.org/en/stable/table/).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating tables

The main class we will use here is called ``Table``:

In [2]:
from astropy.table import Table

Before we look at how to read and write tables, let's first see how to create a table from scratch:

In [48]:
t1 = Table()
t1["name"] = ['source1', "source2", "source3"]
t1["flux"] = [1.1, 1.2, 1.3]

We can look at the table with:

In [49]:
t1

name,flux
str7,float64
source1,1.1
source2,1.2
source3,1.3


We can add columns:

In [50]:
t1['size'] = [1, 2, 6]
t1

name,flux,size
str7,float64,int64
source1,1.1,1
source2,1.2,2
source3,1.3,6


Access the values in a column:

In [51]:
t1["size"]

1
2
6


In [52]:
t1["flux"][1]

1.2

Convert the column to a Numpy array:

In [53]:
import numpy as np
np.array(t1["size"])

array([1, 2, 6])

Access individual cells:

In [54]:
t1[0][0]

'source1'

And access rows:

In [55]:
t1[0]

name,flux,size
str7,float64,int64
source1,1.1,1


## Units in tables

Table columns can include units:

In [56]:
from astropy import units as u
t1["size"].unit = u.cm
t1["flux"].unit = "mJy"
t1

name,flux,size
,mJy,cm
str7,float64,int64
source1,1.1,1
source2,1.2,2
source3,1.3,6


Some unitful operations will then work:

In [57]:
t1['size'].to("m")

<Quantity [0.01, 0.02, 0.06] m>

In [58]:
type(t1['size'].to("m"))

astropy.units.quantity.Quantity

In [59]:
type(t1['size'])

astropy.table.column.Column

However, you may run into unexpected behavior, so if you are planning on using table columns as Quantities, we recommend that you use the ``QTable`` class:

In [60]:
from astropy.table import QTable
qt1 = QTable(t1)

In [61]:
qt1

name,flux,size
,mJy,cm
str7,float64,float64
source1,1.1,1.0
source2,1.2,2.0
source3,1.3,6.0


In [62]:
type(qt1['size'])

astropy.units.quantity.Quantity


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Make a table that contains three columns: <code>spectral type</code>, <code>temperature</code>, and <code>radius</code>, and incude 5 rows with fake data (or real data if you like, for example from <a href="http://www.atlasoftheuniverse.com/startype.html">here</a>). Try including units on the columns that can have them.</li>
<li>Find the mean temperature and the maximum radius</li>
<li>Try and find out how to add and remove rows</li>
<li>Add a new column which gives the luminosity (using $L=4\pi R^2 \sigma T^4$)</li>
</ol>

</div>

</section>


In [73]:
t3 = QTable()
t3["spectType"] = ['source1', "source2", "source3"]
t3["radius"] = [1.1, 1.2, 1.3]
t3["temperature"] = [4000, 5000, 6000]

In [74]:
t3['temperature'].mean()

5000.0

In [75]:
t3["radius"].max()

1.3

In [76]:
import astropy.constants as consts
t3["luminosity"] = 4*np.pi*t3['radius']**2*consts.sigma_sb*t3['temperature']

In [78]:
t3

spectType,radius,temperature,luminosity
,,,W / (K4 m2)
str7,float64,int64,float64
source1,1.1,4000,0.0034487916813248157
source2,1.2,5000,0.0051304339061030305
source3,1.3,6000,0.007225361084428435


## Iterating over tables

It is possible to iterate over rows or over columns. To iterate over rows, iterate over the table itself:

In [63]:
for row in t1:
    print(row)

  name  flux size
        mJy   cm 
------- ---- ----
source1  1.1    1
  name  flux size
        mJy   cm 
------- ---- ----
source2  1.2    2
  name  flux size
        mJy   cm 
------- ---- ----
source3  1.3    6


Rows can act like dictionaries, so you can access specific columns from a row:

In [67]:
for row in t1:
    print(row['size'])

1
2
6


In [70]:
for row in qt1:
    print(row['size'])

1.0 cm
2.0 cm
6.0 cm


Iterating over columns is also easy:

In [69]:
for col in t1.columns:
    print(col)

name
flux
size


In [68]:
for colname in t1.columns:
    column = t1[colname]
    print(column[0])

source1
1.1
1


Accessing specific rows from a column object can be done with the item notation:

In [71]:
t1["size"][0]

1

## Joining tables

The astropy.table sub-package provides a few useful functions for stacking/combining tables. For example, we can do a 'join':

In [94]:
t2 = Table()
t2['name'] = ['source1', 'source3']
t2['flux2'] = [1, 9]

In [95]:
from astropy.table import join

In [96]:
t3 = join(t1, t2, join_type="outer")
t3

name,flux,size,flux2
,mJy,cm,
str7,float64,int64,int64
source1,1.1,1,1
source2,1.2,2,--
source3,1.3,6,9


In [85]:
np.mean(t3["flux2"])

5.0

In [86]:
t3["flux2"].mean()

5.0

## Masked tables

It is possible to mask individual cells in tables:

In [97]:
t4 = Table(masked=True)

In [98]:
t4['id'] = [4, 5, 6]
t4["flux"] = [1.3, 1.5, 1.6]

In [99]:
t4

id,flux
int64,float64
4,1.3
5,1.5
6,1.6


In [100]:
#second flux is the only correct one
# where the mask is true (1) it masks it out else keeps
t4['flux'].mask = [1, 0, 1]
t4

id,flux
int64,float64
4,--
5,1.5
6,--


## Using high-level objects as columns

A few specific astropy high-level objects can be used as columns in table - this includes SkyCoord and Time:

In [104]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [111]:
t6 = Table()
t6["time"] = Time([5000000, 5100000, 5200000], format="mjd")
t6["coord"] = SkyCoord([1, 2, 3]*u.deg, [4, 5, 6]*u.deg)
t6["flux"] = [1, 5, 4] * u.mJy
t6

time,coord,flux
,"deg,deg",mJy
object,object,float64
5000000.0,"1.0,4.0",1.0
5100000.0,"2.0,5.0",5.0
5200000.0,"3.0,6.0",4.0


In [113]:
t6['coord']

<SkyCoord (ICRS): (ra, dec) in deg
    [(1., 4.), (2., 5.), (3., 6.)]>

Note however that you may not necessarily be able to write this table to a file and get it back intact, since being able to store this kind of information is not possible in all file formats.

## Slicing

Tables can be sliced like Numpy arrays:

In [114]:
obs = Table(rows=[('M31' , '2012-01-02', 17.0, 17.5),
                  ('M31' , '2012-01-02', 17.1, 17.4),
                  ('M101', '2012-01-02', 15.1, 13.5),
                  ('M82' , '2012-02-14', 16.2, 14.5),
                  ('M31' , '2012-02-14', 16.9, 17.3),
                  ('M82' , '2012-02-14', 15.2, 15.5),
                  ('M101', '2012-02-14', 15.0, 13.6),
                  ('M82' , '2012-03-26', 15.7, 16.5),
                  ('M101', '2012-03-26', 15.1, 13.5),
                  ('M101', '2012-03-26', 14.8, 14.3)],
            names=['name', 'obs_date', 'mag_b', 'mag_v'])

In [115]:
obs[1:4]

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.1,17.4
M101,2012-01-02,15.1,13.5
M82,2012-02-14,16.2,14.5


In [116]:
obs[obs["mag_b"] > 15.5]

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M82,2012-02-14,16.2,14.5
M31,2012-02-14,16.9,17.3
M82,2012-03-26,15.7,16.5


In [117]:
obs["mag_b", "mag_v"]

mag_b,mag_v
float64,float64
17.0,17.5
17.1,17.4
15.1,13.5
16.2,14.5
16.9,17.3
15.2,15.5
15.0,13.6
15.7,16.5
15.1,13.5



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Starting from the <code>obs</code> table:</p>
<ol>
<li>Make a new table that shows every other row, starting with the second row? (that is, the second, fourth, sixth, etc. rows).</li>
<li>Make a new table the only contains rows where <code>name</code> is <code>M31</code></li>
</ol>

</div>

</section>


## Grouping and Aggregation

It is possible to aggregate rows of a table together - for example, to group the rows by source name in the ``obs`` table, you can do:

In [118]:
obs_by_name = obs.group_by("name")

In [119]:
obs_by_name

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M101,2012-01-02,15.1,13.5
M101,2012-02-14,15.0,13.6
M101,2012-03-26,15.1,13.5
M101,2012-03-26,14.8,14.3
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M31,2012-02-14,16.9,17.3
M82,2012-02-14,16.2,14.5
M82,2012-02-14,15.2,15.5


This is not just sorting the values but actually making it possible to access each group of rows:

In [124]:
for group in obs_by_name.groups:
    print(group, end="\n\n")

name  obs_date  mag_b mag_v
---- ---------- ----- -----
M101 2012-01-02  15.1  13.5
M101 2012-02-14  15.0  13.6
M101 2012-03-26  15.1  13.5
M101 2012-03-26  14.8  14.3

name  obs_date  mag_b mag_v
---- ---------- ----- -----
 M31 2012-01-02  17.0  17.5
 M31 2012-01-02  17.1  17.4
 M31 2012-02-14  16.9  17.3

name  obs_date  mag_b mag_v
---- ---------- ----- -----
 M82 2012-02-14  16.2  14.5
 M82 2012-02-14  15.2  15.5
 M82 2012-03-26  15.7  16.5



We can then aggregate the rows together in each group using a function:

In [129]:
obs_by_name.groups.aggregate(np.mean)

name,mag_b,mag_v
str4,float64,float64
M101,15.000000000000002,13.725000000000001
M31,17.0,17.400000000000002
M82,15.699999999999998,15.5


To skip wasting cycles calculating aggregate functions over all columns we can select which column we want to aggregate over

In [131]:
obs_by_name["name", "mag_v"].groups.aggregate(np.mean)

name,mag_v
str4,float64
M101,13.725000000000001
M31,17.400000000000002
M82,15.5


## Writing data

To write out the data, we can use the ``write`` method:

In [132]:
obs

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M101,2012-01-02,15.1,13.5
M82,2012-02-14,16.2,14.5
M31,2012-02-14,16.9,17.3
M82,2012-02-14,15.2,15.5
M101,2012-02-14,15.0,13.6
M82,2012-03-26,15.7,16.5
M101,2012-03-26,15.1,13.5


In [134]:
obs.write("test2.fits")

In [135]:
obs.write("test2.tex")

In some cases the format will be inferred from the extension, but only in unambiguous cases - otherwise the format has to be specified explicitly:

In [137]:
obs.write("test2.vot", format="votable")

You can find the [list of supported formats](https://docs.astropy.org/en/stable/io/unified.html#built-in-table-readers-writers) in the documentation.

## Reading data

You can also easily read in tables using the ``read`` method:

In [140]:
t7 = Table.read("data/2mass.tbl", format="ascii.ipac")
t7

ra,dec,clon,clat,err_maj,err_min,err_ang,designation,j_m,j_cmsig,j_msigcom,j_snr,h_m,h_cmsig,h_msigcom,h_snr,k_m,k_cmsig,k_msigcom,k_snr,ph_qual,rd_flg,bl_flg,cc_flg,ndet,gal_contam,mp_flg,dist,angle,j_h,h_k,j_k
deg,deg,,,arcsec,arcsec,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,,,,,,,
float64,float64,str12,str13,float64,float64,int64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3,str3,str3,str3,str6,int64,int64,float64,float64,float64,float64,float64
274.429506,-13.870547,18h17m43.08s,-13d52m13.97s,0.08,0.08,45,18174308-1352139,16.305,0.142,0.143,6.7,14.048,0.107,0.108,13.6,13.257,0.066,0.066,16.5,CAA,222,111,0ss,066655,0,0,975.080151,256.448,2.257,0.791,3.048
274.423821,-13.86974,18h17m41.72s,-13d52m11.06s,0.06,0.06,90,18174171-1352110,14.802,0.058,0.059,26.7,12.635,0.059,0.06,50.1,11.768,0.045,0.046,65.2,AAA,222,111,0ss,666666,0,0,993.752042,256.878,2.167,0.867,3.034
274.424587,-13.739629,18h17m41.90s,-13d44m22.66s,0.08,0.08,45,18174190-1344226,16.328,--,--,--,14.345,0.059,0.06,10.4,13.405,0.046,0.047,14.4,UAA,022,011,0cc,003666,0,0,995.726698,284.113,--,0.94,--
274.433933,-13.769502,18h17m44.14s,-13d46m10.21s,0.08,0.08,45,18174414-1346102,16.281,0.098,0.099,6.8,14.057,0.035,0.036,13.5,12.956,0.032,0.033,21.8,CAA,222,111,000,065566,0,0,942.627418,278.252,2.224,1.101,3.325
274.437013,-13.885698,18h17m44.88s,-13d53m08.51s,0.09,0.09,45,18174488-1353085,15.171,--,--,--,14.412,0.152,0.152,9.8,13.742,0.095,0.095,10.6,UBA,622,022,0cc,005566,0,0,964.105389,252.93,--,0.67,--
274.433996,-13.752446,18h17m44.16s,-13d45m08.81s,0.08,0.08,90,18174415-1345088,16.54,--,--,--,14.519,0.083,0.083,8.8,13.604,0.043,0.044,12.0,UBA,022,011,0cc,005666,0,0,953.230532,281.908,--,0.915,--
274.418138,-13.77215,18h17m40.35s,-13d46m19.74s,0.08,0.08,90,18174035-1346197,17.98,--,--,--,14.61,0.043,0.044,8.1,13.456,0.056,0.057,13.8,UBA,022,011,000,001645,0,0,996.047248,277.25,--,1.154,--
274.433695,-13.899049,18h17m44.09s,-13d53m56.58s,0.06,0.06,90,18174408-1353565,13.011,0.021,0.024,139.0,10.917,0.02,0.021,243.8,10.013,0.017,0.019,328.3,AAA,222,111,000,666666,0,0,990.166399,250.466,2.094,0.904,2.998


In [143]:
Table.read.help()

Table.read general documentation

Read and parse a data table and return as a Table.

This function provides the Table interface to the astropy unified I/O
layer.  This allows easily reading a file in many supported data formats
using syntax such as::

  >>> from astropy.table import Table
  >>> dat = Table.read('table.dat', format='ascii')
  >>> events = Table.read('events.fits', format='fits')

Get help on the available readers for ``Table`` using the``help()`` method::

  >>> Table.read.help()  # Get help reading Table and list supported formats
  >>> Table.read.help('fits')  # Get detailed help on Table FITS reader
  >>> Table.read.list_formats()  # Print list of available formats

See also: https://docs.astropy.org/en/stable/io/unified.html

Parameters
----------
*args : tuple, optional
    Positional arguments passed through to data reader. If supplied the
    first argument is typically the input filename.
format : str
    File format specifier.
units : list, dict, optional
    

In [145]:
Table.read.help("ascii")

Table.read(format='ascii') documentation

Read the input ``table`` and return the table.  Most of
the default behavior for various parameters is determined by the Reader
class.

See also:

- https://docs.astropy.org/en/stable/io/ascii/
- https://docs.astropy.org/en/stable/io/ascii/read.html

Parameters
----------
table : str, file-like, list, `pathlib.Path` object
    Input table as a file name, file-like object, list of string[s],
    single newline-separated string or `pathlib.Path` object.
guess : bool
    Try to guess the table format. Defaults to None.
format : str, `~astropy.io.ascii.BaseReader`
    Input table format
Inputter : `~astropy.io.ascii.BaseInputter`
    Inputter class
Outputter : `~astropy.io.ascii.BaseOutputter`
    Outputter class
delimiter : str
    Column delimiter string
comment : str
    Regular expression defining a comment line in table
quotechar : str
    One-character string to quote fields containing special characters
header_start : int
    Line index for 


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Using the <code>t6</code> table above:</p>
<ol>
<li>
<p>Make a plot that shows <code>j_m</code>-<code>h_m</code> on the x-axis, and <code>h_m</code>-<code>k_m</code> on the y-axis</p>
</li>
<li>
<p>Make a new table that contains the subset of rows where the <code>j_snr</code>, <code>h_snr</code>, and <code>k_snr</code> columns, which give the signal-to-noise-ratio in the J, H, and K band, are greater than 10, and try and show these points in red in the plot you just made.</p>
</li>
<li>
<p>Make a new table (based on the full table) that contains only the RA, Dec, and the <code>j_m</code>, <code>h_m</code> and <code>k_m</code> columns, then try and write out this catalog into a format that you can read into another software package. For example, try and write out the catalog into CSV format, then read it into a spreadsheet software package (e.g. Excel, Google Docs, Numbers, OpenOffice). You may run into an issue at this point - if so, take a look at https://github.com/astropy/astropy/issues/7357 to see how to fix it.</p>
</li>
</ol>

</div>

</section>


<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)